In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data
from matrix_factorization import MF, MF_CVIB, MF_IPS, MF_SNIPS, MF_DR
from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "yahoo"

In [2]:
if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from yahoo data set<===
[train] num data: 311704
[test]  num data: 54000
# user: 15401, # item: 1001


In [3]:
"MF CVIB"
mf_cvib = MF_CVIB(num_user, num_item)
mf_cvib.fit(x_train, y_train, 
    lr=0.01,
    batch_size=2048,
    lamb=1e-5,
    alpha=1.0,
    gamma=1e-5,
    tol=1e-6,
    verbose=False)

test_pred = mf_cvib.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_cvib, x_test, y_test)

print("***"*5 + "[MF-CVIB]" + "***"*5)
print("[MF-CVIB] test mse:", mse_mf)
print("[MF-CVIB] test auc:", auc_mf)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-CVIB]" + "***"*5)

[MF-CVIB] epoch:56, xent:85.95468580722809
***************[MF-CVIB]***************
[MF-CVIB] test mse: 0.3811748552008649
[MF-CVIB] test auc: 0.7230968841538437
[MF] ndcg@5:0.713331, ndcg@10:0.807823
Num User: 5400
Gini index: 0.5366045120686594
Global utility: 0.2896666666666667
***************[MF-CVIB]***************


In [4]:
"MF naive"
mf = MF(num_user, num_item)
mf.fit(x_train, y_train, lr=0.01, batch_size=2048, lamb=1e-4,
    verbose=0)
test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf, x_test, y_test)

print("***"*5 + "[MF]" + "***"*5)
print("[MF] test mse:", mse_func(y_test, test_pred))
print("[MF] test auc:", auc_mf)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF]" + "***"*5)

[MF] epoch:39, xent:96.58534151315689
***************[MF]***************
[MF] test mse: 0.24950093735667225
[MF] test auc: 0.6792038830901573
[MF] ndcg@5:0.629765, ndcg@10:0.760142
Num User: 5400
Gini index: 0.5857343739708984
Global utility: 0.2499259259259259
***************[MF]***************


In [5]:
"MF IPS"
mf_ips = MF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_ips.fit(x_train, y_train, y_ips=y_ips,
    lr=0.05, 
    batch_size=2048,
    lamb=1e-4,verbose=0)
test_pred = mf_ips.predict(x_test)
mse_mfips = mse_func(y_test, test_pred)
auc_mfips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_ips, x_test, y_test)

print("***"*5 + "[MF-IPS]" + "***"*5)
print("[MF-IPS] test mse:", mse_func(y_test, test_pred))
print("[MF-IPS] test auc:", auc_mfips)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-IPS]" + "***"*5)

[MF-IPS] epoch:35, xent:2519.3240547180176
***************[MF-IPS]***************
[MF-IPS] test mse: 0.22704478267606706
[MF-IPS] test auc: 0.6827980891431178
[MF] ndcg@5:0.649558, ndcg@10:0.766806
Num User: 5400
Gini index: 0.5550459426895182
Global utility: 0.26185185185185184
***************[MF-IPS]***************


In [6]:
"MF-SNIPS"
mf_snips = MF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_snips.fit(x_train, y_train, y_ips=y_ips,lr=0.05,
        batch_size=2048, lamb=1e-5,
        verbose=0)

test_pred = mf_snips.predict(x_test)
mse_mfsnips = mse_func(y_test, test_pred)
auc_mfsnips = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_snips, x_test, y_test)

print("***"*5 + "[MF-SNIPS]" + "***"*5)
print("[MF-SNIPS] test mse:", mse_mfsnips)
print("[MF-SNIPS] test auc:", auc_mfsnips)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-SNIPS]" + "***"*5)

[MF-SNIPS] epoch:29, xent:59.41869920492172
***************[MF-SNIPS]***************
[MF-SNIPS] test mse: 0.19434143275694046
[MF-SNIPS] test auc: 0.6862716001780234
[MF] ndcg@5:0.645145, ndcg@10:0.768809
Num User: 5400
Gini index: 0.5615337588974333
Global utility: 0.254
***************[MF-SNIPS]***************


In [7]:
"MF DR"
mf_dr = MF_DR(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_dr.fit(x_train, y_train, y_ips=y_ips,lr=0.05, 
    batch_size=2048,
    lamb=1e-4,
    verbose=0)

test_pred = mf_dr.predict(x_test)
mse_mfdr = mse_func(y_test, test_pred)
auc_mfdr = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(mf_dr, x_test, y_test)

print("***"*5 + "[MF-DR]" + "***"*5)
print("[MF-DR] test mse:", mse_mfdr)
print("[MF-DR] test auc:", auc_mfdr)
print("[MF] ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
        np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-DR]" + "***"*5)

[MF-DR] epoch:32, xent:5046561.34375
***************[MF-DR]***************
[MF-DR] test mse: 0.2118318422178481
[MF-DR] test auc: 0.6857209615334211
[MF] ndcg@5:0.657613, ndcg@10:0.774542
Num User: 5400
Gini index: 0.5588956455754206
Global utility: 0.26392592592592595
***************[MF-DR]***************
